# Apache Spark for complex queries

## Data Management Homework 7

In this assignment we will use 
[Apache Spark](https://spark.apache.org/): 
a popular framework for optimal distributed processing on large amount of data. 
The objective of is to use Apache Spark to translate and execute some queries of the TPCx-BB bigdata benchmark.  
[TPCx-BB](https://www.tpc.org/tpc_documents_current_versions/current_specifications5.asp) 
or simply "Big Bench" is a common benchmark suite to evaluate the system performance on big data analytics and machine learning algorithms. 
We will focus on big data analytical queries, which are expressed in SQL. 

Spark is a framework available in multiple languages: Scala, Java, Python, R. In this exercise, we will use Python.

## Setup

### Jupyter Lab

If you are not familiar with the Jupyter Lab environment, check out these resources from the official website: 
[example notebook](https://jupyter.org/try), 
[docs](https://jupyterlab.readthedocs.io/en/stable/getting_started/overview.html). 

Quick reference:
- This is a cell. A cell can contain either Markdown text (such as this one) or code. Everything in jupyter notebook is a cell.
    - Click on the plus on the top bar to add a new cell
    - You can double-click on a text cell to edit iy using Markdown
    - You can run a cell by either using the button "play" at the top bar or by using the "shift + enter" key combination
    - Running a code cell executes it
    - Running a text cell formats the text
- Once you run a cell it stays in memory! So code will be run based on which order you execute cells, even if you execute a cell that is below another one before
- General rule #1: try to arrange cell step-by-stop from top to bottom. If anything breaks, try to execute every cell from the top
- General rule #2: if you are stuck or a cell is blocked during execution re-run the kernel from the top bar menu
 
### Contents
You can navigate through this exercise contents with the file explorer on the left.  
The contents are "extracted" from the 
[TPCx-BB](https://www.tpc.org/tpc_documents_current_versions/current_specifications5.asp) 
benchmark source folder. 
Please refer to the link if you want to have a broader overview and/or additional information TPCx-BB. 
Since this exercise differs from the actual benchmark, only a subset of its content are reported here:
- `queries/` contains 30 SQL/Spark queries, some of which are to be ported to Spark in this exercise. Every query `qxx/` folder (`xx` = number) contains
    - `engineLocalSettings.conf`: TPC related, disregard
    - `engineLocalSettings.sql`: TPC related, disregard
    - `explain_qxx.sql`: *query content* in "explanatory" format
    - `qxx.sql`: *query content* in TPC exec format
    - `run.sh`: TPC related, disregard
    - `results/qxx-result`: contains the expect result in plain-text. You should compare this with your query output (example provided later)
- `spark_table_schemas`: contains schema information for every table in the dataset. Not relevant for the implementation
- `TPCx-BB-dataset`: contains all the tables in separate folder. Refer to it for table names

**Do not modify** `spark_table_schemas` or `TPCx-BB-dataset` contents as it may compromise your solution.

### Guidelines
You must use the Spark SQL module to solve this exercise. Refer to the official documentation:
> Spark SQL: https://spark.apache.org/docs/latest/sql-programming-guide.html

We will work with *DataFrames*: a Spark data type used to represent collections of data, including database Tables. 
You are strongly recommended to refer to the DataFrame API reference within the Spark SQL module during the exercise implementation. 
There you will find methods, functions and further datatypes which are equivalent to SQL operations. 
> DataFrame Reference: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html
The Spark DataFrame API resembles the one of Pandas library.
 
Reference: 
 [PySpark API documentation](https://spark.apache.org/docs/latest/api/python/reference/index.html)

#### Reading TCPx-BB queries
The SQL queries files (`explain_qxx.sql` and `qxx.sql`) are taken directly from the TPCx-BB benchmark suite 
and therefore might contain "extra" SQL statements and comments, 
which are functional to the TCPx-BB original benchmark (e.g. `hive` instructions, `EXPLAIN`, etc.). 
Your goal is to extract and translate the SQL query only, disregarding irrelevant statements/instructions for the purpose of this exercise.  
Additionally, queries might contain *template* variables, in the form `${qxx_variable_name}`. 
You can find all relative templates in the `query/queryParameters.sql` file.

## Environment preparation
**Make sure to read through and run the following code cells before starting the exercise** 

### Install PySpark

Setup of python environment done through `pyproject.toml` specifications and 
[_poetry_](https://python-poetry.org/docs/)
on project root directory.
Follow `README.md` instructions to install the dependencies in a freshly created virtual environment.

### Import PySpark

Whenever working with Spark, you need to either start a Spark Session.
The Spark Session Builder will handle under-the-hood the architecture of the framework discussed in class 
and give us an entry point to programming with Spark .
 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os

# when run locally, spark has one (master) node
spark = SparkSession.builder \
        .master("local") \
        .appName("Homework 07") \
        .getOrCreate()

spark

# PySpark Dataframes

Dataframes are a data structure for data manipulation.
A Dataframe is represented as a 2-dimensional labeled data structure with columns of potentially different types.
Similar to what a spreadsheet or SQL table looks like.
Most functionality and API of 
[Pandas](https://pandas.pydata.org/docs/)
data analysis library is proposed in PySpark with support for distributed collections of data.

To start, let's see how to create a dataframe, visualize the data in it and retrieve its basic properties.

In [43]:
# Creating a DataFrame from scratch
df = spark.createDataFrame(data=[
    (1, "Alice", 34),
    (2, "Bob", 45),
    (3, "Charlie", 56)
], schema=["id", "name", "age"])

# visualize dataframe representation 
df.show() # by default shows first 20 rows

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 34|
|  2|    Bob| 45|
|  3|Charlie| 56|
+---+-------+---+



In [50]:
# some useful methods to explore the table in dataframe
print(f'Type: {type(df)}',    # PySpark Dataframe is not equal to Pandas Dataframe !
      f'First top n=2 rows: {df.head(2)}', # rows returned in a list
      f'Column title names: {df.columns}',
      f'Column title names with its types: {df.dtypes}',
      f'Selecting a column: {df.select("name")}', # it's still a DataFrame
      f'Avoid pandas notation (less features): {df["name"]}', # Column object with lesser features than PySpark DataFrame
      f'Selecting more than one columns: {df.select(["name", "age"])}', # it's still a DataFrame
      sep='\n\n')

Type: <class 'pyspark.sql.dataframe.DataFrame'>

First top n=2 rows: [Row(id=1, name='Alice', age=34), Row(id=2, name='Bob', age=45)]

Column title names: ['id', 'name', 'age']

Column title names with its types: [('id', 'bigint'), ('name', 'string'), ('age', 'bigint')]

Selecting a column: DataFrame[name: string]

Avoid pandas notation (less features): Column<'name'>

Selecting more than one columns: DataFrame[name: string, age: bigint]


In [51]:
# print the values of the selected columns
df.select(["name", "age"]).show()

+-------+---+
|   name|age|
+-------+---+
|  Alice| 34|
|    Bob| 45|
|Charlie| 56|
+-------+---+



In [52]:
# get statistics retrieved from a dataframe, computationally expensive, similar to Pandas
df.describe().show()

+-------+---+-------+----+
|summary| id|   name| age|
+-------+---+-------+----+
|  count|  3|      3|   3|
|   mean|2.0|   NULL|45.0|
| stddev|1.0|   NULL|11.0|
|    min|  1|  Alice|  34|
|    max|  3|Charlie|  56|
+-------+---+-------+----+



## Column operations on DataFrame

The columns operations generate a new DataFrame that need to be stored in a variable to be saved.
It does not change the original DataFrame, but creates a new modified one from it.

Operations on column:
- creation
- renaming
- deletion

In [ ]:
# add a column where age is increase +2
df_col_added = df.withColumn(colName="age in 2 years", col=col("age") + 2)
df_col_added.show()

In [ ]:
# remove the 'id' column 
df_col_removed = df_col_added.drop("id")
df_col_removed.show()

In [ ]:
# rename the 'name' column
df_col_renamed = df_col_removed.withColumnRenamed(existing="name", new="name_column_renamed")
df_col_renamed.show()

## Row operations on DataFrame

The result from this operations need to be saved in a new variable since it does not change the original dataframe. 
Same logic as column operations.

Operations on rows:
- creation
- update
- deletion

Examples provided within the context of handling null missing values in dataframe records.

In [38]:
# update a row setting a null value
df_null = df.withColumn(colName= "age", 
                        col= when(col("age") > 50, None).otherwise(col("age")))
df_null.show()

+---+-------+----+
| id|   name| age|
+---+-------+----+
|  1|  Alice|  34|
|  2|    Bob|  45|
|  3|Charlie|NULL|
+---+-------+----+



Missing values are generally referred as `NA`, which is a sentinel value.
Further explanation in 
[pandas](https://pandas.pydata.org/docs/user_guide/missing_data.html#missing-data),
[pyspark](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.na.html?highlight=na#pyspark.sql.DataFrame.na)
specific docs.

In [39]:
# remove rows with null values in field attributes
df_without_null_rows = df_null.dropna()
# same as df_null.na.drop(), df_null.dropna(how='any'), df_null.na.drop(how='any')
df_without_null_rows.show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|Alice| 34|
|  2|  Bob| 45|
+---+-----+---+



In [41]:
# add a row to dataframe with all null fields in record
df_null_whole_record = df_null.union(spark.createDataFrame([(None, None, None)], schema=df_null.schema))
df_null_whole_record.show()

+----+-------+----+
|  id|   name| age|
+----+-------+----+
|   1|  Alice|  34|
|   2|    Bob|  45|
|   3|Charlie|NULL|
|NULL|   NULL|NULL|
+----+-------+----+



In [42]:
# remove rows if whole record have null field attributes
df_null_without_null_rows = df_null_whole_record.na.drop(how='all')
df_null_without_null_rows.show()

+---+-------+----+
| id|   name| age|
+---+-------+----+
|  1|  Alice|  34|
|  2|    Bob|  45|
|  3|Charlie|NULL|
+---+-------+----+



### Define Helper Function

In [ ]:
def get_table_old(name):
    """
    Load a table from the TPCx-BB dataset.
    The dataframe is read from the Parquet format, 
    which is a columnar storage format ideal for efficient storage and retrieval of data
    :param name: name of the table 
    :return: a Spark DataFrame of the table
    """
    df = spark.read.parquet(f"TPCx-BB-dataset/{name}.ptxt")
    f = open(f"spark_table_schemas/{name}.schema","r")
    lines = f.readlines()
    for line in lines:
        l = line.split()
        if len(l) > 2:
            df.schema[l[0]].nullable = False

    return df

get_table = lambda x : spark.read.option('header', 'true').parquet(f'TPCx-BB-dataset/{x}.ptxt')

### Explore the dataset

You can use `get_table` to load current dataset tables. A table in Spark is stored as a *DataFrame* - see reference in the exercise intro.

In [ ]:
# load the current table 
customer = get_table("customer")
customer.show() 

In [ ]:
# show the 1st row of the customer table
customer.show(1) 

In [ ]:
# display the table schema, which is Spark is a set of [column, type, nullable]
customer.schema 

In [ ]:
# or for a nice pretty print tree view of it
customer.printSchema()

## Sample query translation
Refer to `queries/q00/explain_q00.sql`. The code below is a valid translation of that query using SparkSQL. You can use any methods in the Spark SQL DataFrame class to implement your solution.

### Query 0
Find the amount of items sold by their category.  
Only in certain categories sold in specific stores are considered,


In [58]:
# look in TCPx-BB-dataset to check all the available tables.
# gather tables needed
s = get_table("store_sales")
i = get_table("item")

query0_solution = s.join(i, s.ss_item_sk == i.i_item_sk) \
                .filter(i.i_category_id < 3) \
                .filter(s.ss_store_sk.isin([10, 20, 33, 40, 50])) \
                .groupBy(i.i_category) \
                .count()

query0_solution.show()      

+--------------+-----+
|    i_category|count|
+--------------+-----+
|Home & Kitchen| 1975|
|         Music|25060|
+--------------+-----+



The cell below is a shortcut to display the results file of q00 without navigating to the file.  
The `!` symbol followed by a bash command (`cat` in this case) can be used as in-cell access to the terminal 

In [ ]:
## check the result
!cat queries/q00/results/q00-result

## [YOUR SOLUTION BELOW]
Write the query description in a Markdown cell, followed by a code cell with the query implementation.  
Query descriptions can be found in the TCPx-BB specification, page 93: https://www.tpc.org/tpc_documents_current_versions/current_specifications5.asp

You should implement all the queries assigned in the homework sheet.

### Query X
description

In [ ]:
# implementation

In [ ]:
# check the result